In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "geopandas>1",
#     "lonboard",
#     "palettable",
#     "pandas",
#     "shapely",
#     "sidecar",
# ]
# ///

# Speedtest Data

This example will use data collected from Ookla's Speed Test application and [shared publicly in the AWS Open Data Registry](https://registry.opendata.aws/speedtest-global-performance/). From the AWS page:

> Global fixed broadband and mobile (cellular) network performance, allocated to zoom level 16 web mercator tiles (approximately 610.8 meters by 610.8 meters at the equator). Data is provided in both Shapefile format as well as Apache Parquet with geometries represented in Well Known Text (WKT) projected in EPSG:4326. Download speed, upload speed, and latency are collected via the Speedtest by Ookla applications for Android and iOS and averaged for each tile.

You can view a [hosted version of this notebook on Notebook Sharing Space](https://notebooksharing.space/view/2c2fc0b1ef5f93c70a8c36de30b560d1316d16760714742dcf22a119f4991762#displayOptions=) (35MB download).


## Dependencies

Install [`uv`](https://docs.astral.sh/uv) and then launch this notebook with:

```
uvx juv run examples/internet-speeds.ipynb
```

(The `uvx` command is included when installing `uv`).


## Imports


In [ ]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import shapely
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap

## Fetch data


The URL for a single data file for mobile network speeds in the first quarter of 2019:


In [ ]:
url = "https://ookla-open-data.s3.us-west-2.amazonaws.com/parquet/performance/type=mobile/year=2019/quarter=1/2019-01-01_performance_mobile_tiles.parquet"

The data used in this example is relatively large. In the cell below, we cache the downloading and preparation of the dataset so that it's faster to run this notebook the second time.

We fetch two columns — `avg_d_kbps` and `tile` — from this data file directly from AWS. The `pd.read_parquet` command will perform a network request for these columns from the data file, so it may take a while on a slow network connection. `avg_d_kbps` is the average download speed for that data point in kilobits per second. `tile` is the WKT string representing a given zoom-16 Web Mercator tile.

The `tile` column contains _strings_ representing WKT-formatted geometries. We need to parse those strings into geometries. Then for simplicity we'll convert into their centroids.


In [ ]:
local_path = Path("internet-speeds.parquet")
if local_path.exists():
    gdf = gpd.read_parquet(local_path)
else:
    columns = ["avg_d_kbps", "tile"]
    df = pd.read_parquet(url, columns=columns)

    tile_geometries = shapely.from_wkt(df["tile"])
    tile_centroids = shapely.centroid(tile_geometries)
    gdf = gpd.GeoDataFrame(df[["avg_d_kbps"]], geometry=tile_centroids, crs="EPSG:4326")
    gdf.to_parquet(local_path)

We can take a quick look at this data:


In [ ]:
gdf.head()

To ensure that this demo is snappy on most computers, we'll filter to a bounding box over Europe.

If you're on a recent computer, feel free to comment out the next line.


In [ ]:
gdf = gdf.cx[-11.83:25.5, 34.9:59]

Even this filtered data frame still has 800,000 rows, so it's still a lot of data to explore:


In [ ]:
gdf

To render point data, first create a `ScatterplotLayer` and then add it to a `Map` object:


In [ ]:
sidecar = Sidecar()

In [ ]:
layer = ScatterplotLayer.from_geopandas(gdf)
m = Map(layer, height=800)
with sidecar:
    display(m)

We can look at the [documentation for `ScatterplotLayer`](https://developmentseed.org/lonboard/latest/api/layers/scatterplot-layer/) to see what other rendering options it allows. Let's set the fill color to something other than black:


In [ ]:
layer.get_fill_color = [0, 0, 200, 200]

Blue is pretty, but the map would be more informative if we colored each point by a relevant characteristic. In this case, we have the download speed associated with each location, so let's use that!


Here we compute a linear statistic for the download speed. Given a minimum bound of `5000` and a maximum bound of `50,000` the normalized speed is linearly scaled to between 0 and 1.


In [ ]:
min_bound = 5000
max_bound = 50000
download_speed = gdf["avg_d_kbps"]
normalized_download_speed = (download_speed - min_bound) / (max_bound - min_bound)

`normalized_download_speed` is now linearly scaled based on the bounds provided above. Keep in mind that the **input range of the colormap is 0-1**. So any values that are below 0 will receive the left-most color in the colormap, while any values above 1 will receive the right-most color in the colormap.


In [ ]:
normalized_download_speed

We can use any colormap provided by the [`palettable`](https://github.com/jiffyclub/palettable) package. Let's inspect the `BrBG_10` diverging colormap below:


In [ ]:
BrBG_10.mpl_colormap

Now let's apply the colormap on `normalized_download_speed` using a helper provided by `lonboard`. We can set it on `layer.get_fill_color` to update the existing colors.


In [ ]:
layer.get_fill_color = apply_continuous_cmap(
    normalized_download_speed,
    BrBG_10,
    alpha=0.7,
)

After running the above cell, you should see the map above update with a different color per point!


We can pass an array into any of the "accessors" supported by the layer (this is any attribute that starts with `get_*`).

For demonstration purposes, let's also set `get_radius` to `normalized_download_speed`.


In [ ]:
layer.get_radius = normalized_download_speed * 200
layer.radius_units = "meters"
layer.radius_min_pixels = 0.5

After running the above cell, you should see the map updated to have a different radius per point!
